# Lab 04-1 Multivariable Linear Regression

In [27]:
import torch
from torch import optim
import math
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

In [18]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [19]:
# compute H(x)
# hypothesis = x1_train * w1 + x2_train * w2 _ x3_train * w3 + b
# hypothesis = x_train.matmul(W) + b
# cost = torch.mean((hypotheis - y_train) ** 2)

# Gradient descent with torch.optim

# optimizer = optim.SGD([W, b], lr = 1e-5)
# optimizer.zero_grad()
# cost.backward()
# optimizer.step()

In [24]:
# initialize model
W = torch.zeros((3, 1), requires_grad= True)
b = torch.zeros(1, requires_grad= True)

# setting optimizer
optimizer = optim.SGD([W, b], lr = 1e-5)

In [33]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):

  # compute H(x)
  hypothesis = x_train.matmul(W) + b
  # compute cost
  cost = torch.mean((hypothesis - y_train) ** 2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d} /{} hypothesis: {} cost: {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))


Epoch    0 /20 hypothesis: tensor([152.8021, 183.6749, 180.9686, 197.0709, 140.1016]) cost: 1.617650
Epoch    1 /20 hypothesis: tensor([152.8019, 183.6754, 180.9687, 197.0710, 140.1022]) cost: 1.616944
Epoch    2 /20 hypothesis: tensor([152.8016, 183.6758, 180.9687, 197.0710, 140.1027]) cost: 1.616214
Epoch    3 /20 hypothesis: tensor([152.8012, 183.6762, 180.9686, 197.0710, 140.1032]) cost: 1.615513
Epoch    4 /20 hypothesis: tensor([152.8008, 183.6765, 180.9686, 197.0710, 140.1036]) cost: 1.614820
Epoch    5 /20 hypothesis: tensor([152.8004, 183.6768, 180.9684, 197.0709, 140.1041]) cost: 1.614109
Epoch    6 /20 hypothesis: tensor([152.8000, 183.6772, 180.9683, 197.0708, 140.1045]) cost: 1.613393
Epoch    7 /20 hypothesis: tensor([152.7995, 183.6775, 180.9682, 197.0706, 140.1049]) cost: 1.612701
Epoch    8 /20 hypothesis: tensor([152.7991, 183.6778, 180.9681, 197.0705, 140.1053]) cost: 1.611981
Epoch    9 /20 hypothesis: tensor([152.7987, 183.6781, 180.9680, 197.0704, 140.1057]) cost:

In [37]:
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

model = MultivariateLinearRegressionModel()
hypothesis = model(x_train)

# setting optimizer
optimizer = optim.SGD(model.parameters(), lr = 1e-5)

In [38]:
import torch.nn.functional as F

nb_epochs = 20
for epoch in range(nb_epochs + 1):

  # compute H(x)
  hypothesis = model(x_train)
  # compute cost
  cost = F.mse_loss(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d} /{} hypothesis: {} cost: {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))


Epoch    0 /20 hypothesis: tensor([-27.8224, -29.5243, -31.2161, -32.7061, -22.6053]) cost: 40474.074219
Epoch    1 /20 hypothesis: tensor([50.7449, 64.9077, 61.8294, 68.6181, 49.4226]) cost: 12687.152344
Epoch    2 /20 hypothesis: tensor([ 94.7319, 117.7767, 113.9222, 125.3457,  89.7484]) cost: 3977.428223
Epoch    3 /20 hypothesis: tensor([119.3586, 147.3761, 143.0870, 157.1055, 112.3253]) cost: 1247.391357
Epoch    4 /20 hypothesis: tensor([133.1462, 163.9478, 159.4153, 174.8866, 124.9653]) cost: 391.670105
Epoch    5 /20 hypothesis: tensor([140.8654, 173.2256, 168.5570, 184.8415, 132.0420]) cost: 123.446548
Epoch    6 /20 hypothesis: tensor([145.1871, 178.4199, 173.6751, 190.4149, 136.0040]) cost: 39.372684
Epoch    7 /20 hypothesis: tensor([147.6067, 181.3280, 176.5405, 193.5352, 138.2221]) cost: 13.020105
Epoch    8 /20 hypothesis: tensor([148.9613, 182.9561, 178.1447, 195.2822, 139.4640]) cost: 4.759938
Epoch    9 /20 hypothesis: tensor([149.7197, 183.8677, 179.0429, 196.2602, 1

## Lab 04-2 Loading data

In [40]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                  [93, 88, 93],
                  [89, 91, 90],
                  [96, 98, 100],
                  [73, 66, 70]]
    self.y_data =  [[152], [185], [180], [196], [142]]

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])

    return x, y


dataset = CustomDataset()

In [41]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size = 2,
    shuffle = True
)

In [42]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  # get index, data of minibatch
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    # compute H(x)
    prediction = model(x_train)

    # compute cost
    cost = F.mse_loss(prediction, y_train)

    # upgrade H(x)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
    ))
    # len(dataloader) : the number of minibatch per epoch

Epoch    0/20 Batch 1/3 Cost: 1.995291
Epoch    0/20 Batch 2/3 Cost: 0.007015
Epoch    0/20 Batch 3/3 Cost: 1.095788
Epoch    1/20 Batch 1/3 Cost: 0.154398
Epoch    1/20 Batch 2/3 Cost: 1.163248
Epoch    1/20 Batch 3/3 Cost: 4.240259
Epoch    2/20 Batch 1/3 Cost: 2.009468
Epoch    2/20 Batch 2/3 Cost: 0.120577
Epoch    2/20 Batch 3/3 Cost: 1.365613
Epoch    3/20 Batch 1/3 Cost: 0.361470
Epoch    3/20 Batch 2/3 Cost: 2.209672
Epoch    3/20 Batch 3/3 Cost: 0.041813
Epoch    4/20 Batch 1/3 Cost: 0.962557
Epoch    4/20 Batch 2/3 Cost: 1.774580
Epoch    4/20 Batch 3/3 Cost: 0.008452
Epoch    5/20 Batch 1/3 Cost: 0.517462
Epoch    5/20 Batch 2/3 Cost: 2.050797
Epoch    5/20 Batch 3/3 Cost: 0.002810
Epoch    6/20 Batch 1/3 Cost: 1.053666
Epoch    6/20 Batch 2/3 Cost: 0.967310
Epoch    6/20 Batch 3/3 Cost: 1.983853
Epoch    7/20 Batch 1/3 Cost: 0.598451
Epoch    7/20 Batch 2/3 Cost: 3.063670
Epoch    7/20 Batch 3/3 Cost: 0.018525
Epoch    8/20 Batch 1/3 Cost: 0.006555
Epoch    8/20 Batch 2/3 C